Name: Md Abdul Kadir

Linear Model


# Section: Dataset and model class implementation

In [182]:
from sklearn.model_selection import train_test_split

# dataset class
class NutritionData:
    
    def __init__(self, data_path=None):
        self.GOOD_OUTPUT =['Animal Products', 'Animal fats', 'Meat', 'Cereals - Excluding Beer','Vegetables','Vegetal Products','Vegetable Oils', 'Oilcrops' ]
        self.NOT_IPMORTANT_COLUMNS  = ["Country", 'Unit (all except Population)', 'Population', 
                                      'Obesity', 'Undernourished', 'Confirmed', 'Deaths','Recovered', 'Active',
                                      'Alcoholic Beverages',
           'Aquatic Products, Other', 'Eggs',
           'Fish, Seafood', 'Fruits - Excluding Wine',
           'Milk - Excluding Butter', 'Offals','Pulses', 'Spices',
           'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
           'Treenuts',
           'Miscellaneous']
        
        if data_path != None:
            self.data_path = data_path
            dataframe = pd.read_csv(self.data_path)
            dataframe = self.__replaceNaN(dataframe, replaceby="mostfrq")

            ##Standard Scaler 
            #NOT_IPMORTANT_COLUMNS = ["Country", 'Unit (all except Population)', 'Population', 
            #                       'Obesity', 'Undernourished', 'Confirmed', 'Deaths','Recovered', 'Active']

            

            clm = dataframe.columns.drop(self.NOT_IPMORTANT_COLUMNS)

            dataframe = self.__replace_char(dataframe, "Undernourished", char=r'^<')
            scaled_df, self.scaler   = self.__scale(dataframe, clm, scaler="mm")
            scaled_df = scaled_df.sample(frac=1,random_state=1).reset_index(drop=True)

            self.dataset = scaled_df
        
    
        # Importan Methodes
    # Replace the nan vale with mean, median, or most frequent value
    def __replaceNaN(self, dataframe, replaceby = "mean"):
        if (replaceby == "mean"):
            return dataframe.fillna(dataframe.mean())

        elif (replaceby == "median"):
            return dataframe.fillna(dataframe.median())

        elif (replaceby == "mostfrq"):
            return dataframe.fillna(dataframe.mode().iloc[0])

        else:return dataframe.dropna()
        
        

    # Replce any nonnumaric char from a column.    
    def __replace_char(self, dataframe, collumn, char=r'^<'):
        
        dataframe[collumn] =  dataframe[collumn].replace(regex=char, value='')
        dataframe[collumn] = pd.to_numeric(dataframe[collumn],errors='coerce')
        return dataframe



    # Scale the data using standard scaler or max min scaler. Menstion the names to scale. 
    def __scale(self, dataframe, coulumns, scaler="ss"):
        
        if (scaler == "mm"):
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
            scaler.fit(dataframe[coulumns])
            dataframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe, scaler
        elif (scaler == "ss"):
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            scaler.fit(dataframe[coulumns])
            datqframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
            return dataframe, scaler

        else:return dataframe, scaler





In [183]:
import pickle
import logging
import os

In [184]:
# Generalizing class to test and train all the models
class NutritionModel:
    
    def __init__(self, dataset=None):
        self.dataset = dataset
        self.model = None
        self.train_set = None
        self.test_set = None
        
        
    def split_data(self, test_size=0.2, random_state=42):
        # splits the dataset in train and test set
        self.train_set, self.test_set  = train_test_split(self.dataset, test_size=test_size, random_state=42)
      
    
    def set_model(self, model):
        # setting up the model
        self.model = model
        
        
    def save_model(self, pkl_filename='model.pkl'):
        with open(pkl_filename, 'wb') as file:
            pickle.dump(self.model, file)
    
    
    
    def load_model(self, pkl_filename='model.pkl'):
        with open(pkl_filename, 'rb') as file:
            self.model = pickle.load(file)
    
        
    
    def train_model(self, x_column, y_column):
        #traing the model
        # input arg: x column name, y column names
        self.model.fit(self.train_set[x_column], self.train_set[y_column])
    
    
    def test_model(self, x_column, y_column): 
        score = self.model.score(self.test_set[x_column], self.test_set[y_column])
        return score
    
    
    def predict(self, X):     
        return self.model.predict(X)
    

In [185]:
FEATURE_CLS = ['Obesity', 'Undernourished', 'Deaths','Recovered','Active','Confirmed']
FEATURE = ['Obesity', 'Undernourished', 'Deaths','Recovered' ]

ALL_OUTPUT = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Miscellaneous']
#GOOD_OUTPUT=['Animal Products', 'Cereals - Excluding Beer', 'Oilcrops', 'Meat','Animal fats','Vegetal Products','Vegetable Oils' ]

In [186]:

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
print(rf.get_params())
#Parameters currently in use:


Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [187]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)



# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1, refit=True)
# Fit the random search model

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [188]:
# Tareque 

# Create class to that take a csv file as an input
# Return output prediction 

from sklearn import linear_model

dataobj = NutritionData(data_path=data_path) # creating dataset
# creating model classes
model = NutritionModel(dataset=dataobj.dataset)

model.set_model(model=rf_random)
model.split_data()

x_val = FEATURE
model.train_model(x_column=x_val, y_column=dataobj.GOOD_OUTPUT)
score = model.test_model(x_column=x_val, y_column=dataobj.GOOD_OUTPUT)

print(score)

        
        
    
    


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.1s finished


-0.022024923937873753


In [189]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

method_names = {}
# Adding linear regression
LinearREG = LinearRegression()
method_names["Linear Regression"] = LinearREG

# Adding reage regression 
params_Ridge = {'alpha': [1, 0.1,0.01,0.001,0.0001,0] ,
                "fit_intercept": [True, False],
                "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}


LinearREGRedgeCV = RandomizedSearchCV(estimator = Ridge(),
                                      param_distributions = params_Ridge,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)

method_names["Redge"] = LinearREGRedgeCV


# Adding Lasso regression

params_Lasso = {'alpha': [1, 0.1,0.01,0.001,0.0001,0] ,
                "fit_intercept": [True, False],
               }
LinearREGLassoCV =  RandomizedSearchCV(estimator = Lasso(),
                                      param_distributions = params_Lasso,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)



method_names["Lasso"] = LinearREGLassoCV


# Adding SVR


params_SVR = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

SupportVectorRegression =  RandomizedSearchCV(estimator = SVR(),
                                      param_distributions = params_SVR,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)



method_names["SVR"] = SupportVectorRegression


# Adding Decission Tree
params_decission_tree = [{'criterion': ['mse', 'friedman_mse', 'mae'],
                          'ccp_alpha': [0.0 , 1e-3, 1e-4, 0.5, 1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [1,2,3,4,],
                         'random_state':[1,4,5,100]},
                         {'criterion': [ 'friedman_mse'],
                          'ccp_alpha': [1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [10,11,12,13,14],
                         'random_state':[1,4,5,100]},
                         {'criterion': [ 'mae'],
                          'ccp_alpha': [0.0 , 1e-3, 1e-4, 0.5, 1.0],
                          'max_features' : [1,2,3,4,5],
                         'max_depth': [5,6,7,8,9],
                         'random_state':[100]},
                         
                         ]

#print(DecisionTreeRegressor().get_params().keys())

DecissionTree =  RandomizedSearchCV(estimator = DecisionTreeRegressor(),
                                      param_distributions = params_decission_tree,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)


method_names["DecissionTree"] = DecissionTree

#  Adding RandomForestRegressor


params_random_forest = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

RandomForest =  RandomizedSearchCV(estimator = RandomForestRegressor(),
                                      param_distributions = params_random_forest,
                                      n_iter = 10,
                                      cv = 5, verbose=2,
                                      random_state=42,
                                      n_jobs = -1, refit=True)


method_names["RandomForest"] = RandomForest




In [190]:
# data_file: Adrres of csv
# methods : dictionary of methods (method_names) 
def search_models(data_file, methods, features, predictors):
    dataobj = NutritionData(data_path=data_file)
    model = NutritionModel(dataset=dataobj.dataset)
    model.split_data()
    x_val = features
    GOOD_OUTPUT = predictors
    r2_score = []
    for key, value in method_names.items():
        model.set_model(model=value)
        if key == 'SVR':
            score = 0
            for item in predictors:
                model.train_model(x_column=x_val, y_column=item)
                score += model.test_model(x_column=x_val, y_column=item)
                #print(score/len(GOOD_OUTPUT))
            r2_score.append(score/len(predictors))
            
        
        else:
            model.train_model(x_column=x_val, y_column=predictors)
            score = model.test_model(x_column=x_val, y_column=predictors)
            r2_score.append(score)
            #print(score)


        
    data_frame_dict = {'Methods': list(method_names.keys()),
                   'Input':[FEATURE for i in range(len(method_names.keys()))],
                   'Output':[GOOD_OUTPUT for i in range(len(method_names.keys()))], 
                   'R^2 Score': r2_score }

    df = pd.DataFrame(data=data_frame_dict)
    print(df.head)
    return df

    

In [191]:
# Choose the best model for each of four dataset

paths = [os.path.join("../Data/updated_07_06_2020", "Food_Supply_Quantity_kg_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Fat_Supply_Quantity_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Food_Supply_kcal_Data.csv" ),
        os.path.join("../Data/updated_07_06_2020", "Protein_Supply_Quantity_Data.csv" )]



output = NutritionData().GOOD_OUTPUT
for path in paths:
    protein_prdiction = search_models(path,method_names,FEATURE, output)
    output_path = os.path.join('../Data/outputfiles_reduced', os.path.split(path)[-1])
    protein_prdiction.to_csv(output_path)





Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  50 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   30.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.6s finished


<bound method NDFrame.head of              Methods                                         Input  \
0  Linear Regression  [Obesity, Undernourished, Deaths, Recovered]   
1              Redge  [Obesity, Undernourished, Deaths, Recovered]   
2              Lasso  [Obesity, Undernourished, Deaths, Recovered]   
3                SVR  [Obesity, Undernourished, Deaths, Recovered]   
4      DecissionTree  [Obesity, Undernourished, Deaths, Recovered]   
5       RandomForest  [Obesity, Undernourished, Deaths, Recovered]   

                                              Output  R^2 Score  
0  [Animal Products, Animal fats, Meat, Cereals -...   0.200632  
1  [Animal Products, Animal fats, Meat, Cereals -...   0.172591  
2  [Animal Products, Animal fats, Meat, Cereals -...   0.169464  
3  [Animal Products, Animal fats, Meat, Cereals -...   0.157113  
4  [Animal Products, Animal fats, Meat, Cereals -...   0.143764  
5  [Animal Products, Animal fats, Meat, Cereals -...   0.206798  >


In [ ]:

#Store the best model for all datasets

def store_models(data_file, model_name, method_name, features, predictors, output_dir):
    dataobj = NutritionData(data_path=data_file)
    model = NutritionModel(dataset=dataobj.dataset)
    model.split_data()
    x_val = features
    GOOD_OUTPUT = predictors
    model.set_model(model=model_name)
    model_file = os.path.join(output_dir, os.path.split(data_file)[-1].split('.')[0] + '.pkl')

    if method_name == 'SVR':
        score = 0
        for item in predictors:
            model.train_model(x_column=x_val, y_column=item)
            score += model.test_model(x_column=x_val, y_column=item)
            print(score/len(predictors))
            
            
        
    else:
        model.train_model(x_column=x_val, y_column=predictors)
        score = model.test_model(x_column=x_val, y_column=predictors)
        ideal_prediction = model.predict([[0,0,0,1],])
        ideal_prediction = dataobj.scaler.inverse_transform(ideal_prediction)
        model.save_model(model_file)
        print(score)
        
    

    return (os.path.split(data_file)[-1].split('.')[0], model_file, ideal_prediction)





In [155]:
# Four models for four datasets
model_list =[]
for path in paths:
    model_address  = store_models(path,method_names["RandomForest"],"RandomForest", FEATURE, output, '../Data/outputfiles_reduced/models')
    model_list.append(model_address)
    



Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.6s finished


ValueError: operands could not be broadcast together with shapes (1,7) (8,) (1,7) 

In [86]:
# Prdict output

nutrition_category = []
food_consumtion = []


for model in model_list:
    
    nutrition_category.append(model[0])
    #nutrition_model = NutritionModel()
    #nutrition_model.load_model(model[1])
    food_consumtion.append(list(model[2][0]))
    
print(food_consumtion)



    
df_dict = {'Nutrition Category': nutrition_category,
           str(FEATURE):[[0,0,0,1] for i in range(4)],
           str(ooutput): food_consumtion
          }

df = pd.DataFrame(data= df_dict)
df.to_csv(os.path.join('../Data/outputfiles', 'ideal_food_consumtion.csv'))
    


[[3.261095347453607, 0.5329404672301871, 4.207428179801412, 0.13791637435851045, 10.658460749013324, 0.5624469485995601, 2.450844151444623, 6.067777304138026, 4.298095257579506, 3.814721057125187, 0.6398104946139958, 0.15284596217941673, 1.1707663398327481, 0.6623256566306277, 0.07436803616175217, 4.748302788457057, 0.07779653219007089, 3.03231451989898, 0.4957354267294556, 0.4596981691884802, 0.8940779285311996, 7.431964064700125, 28.950063390031055], [0.0082264428016428, 22.67102909641221, 3.570607155915938, 0.0049350189127049405, 3.9780535205712195, 1.2228166019726598, 1.7765407176365324, 0.49463392608694423, 11.794091900212168, 0.10303062833270141, 3.8743494033815047, 0.09928907567113021, 1.791557568288561, 0.4084227259983803, 0.18452021151234035, 0.551611737539474, 0.18098144255610046, 0.007675940693057114, 0.014303851434704118, 2.22077587877375, 24.22802178791316, 15.009182966198178, 0.28049252939975533], [1.550231608830507, 10.65570929200709, 1.0858511758127498, 0.04112379708775

In [152]:
a = [[1,3] *5]
print(a)

[[1, 3, 1, 3, 1, 3, 1, 3, 1, 3]]
